# GRPO Fine-Tuning for Financial Reasoning

This notebook implements a **training pipeline** for financial reasoning using GRPO (Group Relative Policy Optimization).

## Training Pipeline

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  1. Load Base   │ -> │ 2. Bootstrap    │ -> │ 3. SFT Phase    │ -> │ 4. GRPO Phase   │
│  Qwen2.5-7B     │    │ Data (Gemini)   │    │ (OPTIONAL)      │    │ (RL Fine-tune)  │
└─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
```

## Key Features

1. **Model**: Qwen2.5-7B-Instruct (good math/reasoning priors)
2. **Higher LoRA Rank**: r=64 for complex reasoning tasks
3. **Bootstrapping**: Gemini generates synthetic reasoning traces (optional)
4. **SFT Phase**: SKIPPED by default - base model already has ~80% format compliance
5. **GRPO Phase**: Reinforcement learning to improve accuracy

## Verbose Evaluation

Evaluation outputs show **full model responses** (no truncation) including:
- Complete question
- Ground truth answer
- Full model response with thinking and answer
- Extracted thinking and answer
- Accuracy and format compliance per question

## Configuration Options

| Option | Default | Description |
|--------|---------|-------------|
| `cleanup_old_checkpoints` | `False` | Delete old SFT/GRPO checkpoints before starting |
| `cleanup_bootstrap_data` | `False` | Regenerate bootstrap data from scratch |
| `skip_sft` | `True` | Skip SFT phase (recommended - base model has good format compliance) |

## Requirements

- NVIDIA GPU with CUDA
- `.env` file with `HF_TOKEN` and `GEMINI_API_KEY`


## Phase 1: Setup & Configuration


In [1]:
import os
import re
import json
import random
import shutil
import warnings
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import polars as pl
from datasets import load_dataset, Dataset
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Project paths
PROJECT_ROOT = Path().absolute().parent.parent
MODELS_DIR = PROJECT_ROOT / "models"
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = MODELS_DIR / "checkpoints" / "grpo-financial-v2"
BOOTSTRAP_FILE = DATA_DIR / "processed" / "bootstrap_data.json"

# Create directories
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(DATA_DIR / "processed").mkdir(parents=True, exist_ok=True)

# Load environment
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path if env_path.exists() else None)

# =============================================================================
# CONFIGURATION - Enhanced for better reasoning
# =============================================================================
CONFIG = {
    # ==========================================================================
    # CLEANUP - Set to True to delete old checkpoints before starting fresh
    # ==========================================================================
    "cleanup_old_checkpoints": False,  # True = delete old SFT/GRPO checkpoints
    "cleanup_bootstrap_data": False,   # True = regenerate bootstrap data from scratch
    
    # Model - Upgraded to 7B Instruct for better reasoning
    "model_name": "Qwen/Qwen2.5-7B-Instruct",
    "max_seq_length": 8192,
    "load_in_4bit": True,
    
    # LoRA - Higher rank for complex reasoning
    "lora_r": 64,
    "lora_alpha": 64,
    "lora_dropout": 0,
    
    # Bootstrap Data Generation
    "bootstrap_samples": 500,
    "gemini_model": "gemini-2.5-pro",
    
    # SFT Phase - Skipped by default since base model already has good format compliance
    # Set to False if bootstrap data quality is high and you want to pre-train on it
    "skip_sft": True,  # True = skip SFT, go directly from base model to GRPO
    "sft_epochs": 1,
    "sft_learning_rate": 2e-5,
    "sft_batch_size": 1,
    "sft_gradient_accumulation": 4,
    
    # GRPO Phase
    "grpo_learning_rate": 5e-6,
    "grpo_max_steps": 200,
    "grpo_num_generations": 4,
    "grpo_max_completion_length": 2048,  # Longer for full reasoning traces
    
    # Evaluation
    "eval_samples": 20,
    "random_seed": 42,
    "max_new_tokens": 1024,
}

# Validate API keys
HF_TOKEN = os.getenv("HF_TOKEN", "")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")

print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"\n📦 Model: {CONFIG['model_name']}")
print(f"   LoRA Rank: {CONFIG['lora_r']}")
print(f"   Max Seq Length: {CONFIG['max_seq_length']}")
print(f"\n📊 Bootstrap Samples: {CONFIG['bootstrap_samples']}")
print(f"   Gemini Model: {CONFIG['gemini_model']}")
sft_status = "SKIPPED" if CONFIG['skip_sft'] else f"{CONFIG['sft_epochs']} epoch(s)"
print(f"\n🎓 SFT: {sft_status}")
print(f"   GRPO: {CONFIG['grpo_max_steps']} steps")
print(f"\n🔑 HF Token: {'✓' if HF_TOKEN else '✗'}")
print(f"   Gemini API Key: {'✓' if GEMINI_API_KEY else '✗'}")
print("="*60)


CONFIGURATION

📦 Model: Qwen/Qwen2.5-7B-Instruct
   LoRA Rank: 64
   Max Seq Length: 8192

📊 Bootstrap Samples: 500
   Gemini Model: gemini-2.5-pro

🎓 SFT: SKIPPED
   GRPO: 200 steps

🔑 HF Token: ✓
   Gemini API Key: ✓


In [2]:
# =============================================================================
# CLEANUP PHASE - Remove old checkpoints if requested
# =============================================================================
# This ensures a completely fresh start (the model is ALWAYS loaded fresh from
# HuggingFace regardless, but this cleans up old saved checkpoints from disk)

def cleanup_old_runs():
    """Clean up old checkpoints and optionally bootstrap data."""
    cleaned = []
    
    if CONFIG["cleanup_old_checkpoints"]:
        # Clean up SFT checkpoint
        sft_dir = OUTPUT_DIR / "sft-checkpoint"
        if sft_dir.exists():
            shutil.rmtree(sft_dir)
            cleaned.append(f"SFT checkpoint: {sft_dir}")
        
        # Clean up final model
        final_dir = OUTPUT_DIR / "final-model"
        if final_dir.exists():
            shutil.rmtree(final_dir)
            cleaned.append(f"Final model: {final_dir}")
        
        # Clean up any GRPO checkpoints
        for checkpoint_dir in OUTPUT_DIR.glob("checkpoint-*"):
            if checkpoint_dir.is_dir():
                shutil.rmtree(checkpoint_dir)
                cleaned.append(f"GRPO checkpoint: {checkpoint_dir}")
        
        # Clean up comparison results
        results_csv = OUTPUT_DIR / "comparison_results.csv"
        if results_csv.exists():
            results_csv.unlink()
            cleaned.append(f"Results CSV: {results_csv}")
    
    if CONFIG["cleanup_bootstrap_data"]:
        if BOOTSTRAP_FILE.exists():
            BOOTSTRAP_FILE.unlink()
            cleaned.append(f"Bootstrap data: {BOOTSTRAP_FILE}")
    
    return cleaned

# Run cleanup
cleaned_items = cleanup_old_runs()

if cleaned_items:
    print("🧹 CLEANUP COMPLETED:")
    for item in cleaned_items:
        print(f"  ✓ Deleted: {item}")
    print()
else:
    if CONFIG["cleanup_old_checkpoints"] or CONFIG["cleanup_bootstrap_data"]:
        print("🧹 Cleanup requested but nothing to clean up")
    else:
        print("ℹ️  Cleanup not requested (set cleanup_old_checkpoints=True to clean up)")

# Show what exists
print("\n📁 Current state of output directory:")
if OUTPUT_DIR.exists():
    for item in sorted(OUTPUT_DIR.iterdir()):
        size = sum(f.stat().st_size for f in item.rglob('*') if f.is_file()) / (1024*1024) if item.is_dir() else item.stat().st_size / (1024*1024)
        print(f"  {'📁' if item.is_dir() else '📄'} {item.name}: {size:.1f} MB")
else:
    print("  (empty)")

if BOOTSTRAP_FILE.exists():
    size = BOOTSTRAP_FILE.stat().st_size / (1024*1024)
    print(f"\n📄 Bootstrap data exists: {BOOTSTRAP_FILE.name} ({size:.1f} MB)")
else:
    print("\n📄 Bootstrap data: not generated yet")


ℹ️  Cleanup not requested (set cleanup_old_checkpoints=True to clean up)

📁 Current state of output directory:
  📁 sft-checkpoint: 1575.7 MB

📄 Bootstrap data exists: bootstrap_data.json (5.2 MB)


## Phase 2: Load & Prepare Dataset


In [3]:
# Load ConvFinQA dataset
print("Loading ConvFinQA dataset...")
dataset = load_dataset("FinGPT/fingpt-convfinqa", split="train")
df = pl.from_pandas(dataset.to_pandas())
print(f"✓ Loaded {len(df):,} samples")

# System prompt for financial reasoning (used for both SFT and GRPO)
# NOTE: No code generation - model should reason and answer directly
SYSTEM_PROMPT = """You are an expert financial analyst. Answer financial questions accurately using step-by-step reasoning.

Follow this exact format:

1. First, think through the problem step-by-step in <think> tags
2. Then provide your final answer in <answer> tags

Example:
<think>
The question asks for percentage change from $100M to $150M revenue.
Formula: ((new - old) / old) * 100
- Old value: 100
- New value: 150  
- Calculation: ((150 - 100) / 100) * 100 = 50%
</think>

<answer>50%</answer>"""

def parse_input(input_text: str) -> tuple:
    """Parse ConvFinQA input into context and question."""
    if "Question:" in input_text:
        parts = input_text.split("Question:", 1)
        context = parts[0].replace("Context:", "").strip()
        question = parts[1].strip()
    else:
        context = input_text
        question = input_text.split(".")[-1].strip()
    return context, question

def format_sample(row) -> dict:
    """Format a sample for training."""
    context, question = parse_input(row["input"])
    return {
        "context": context,
        "question": question,
        "answer": str(row["output"]),
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
        ],
    }

# Process all samples
print("Formatting samples...")
processed = [format_sample(row) for row in df.to_pandas().to_dict('records')]
processed_df = pd.DataFrame(processed)
print(f"✓ Processed {len(processed_df):,} samples")


Loading ConvFinQA dataset...
✓ Loaded 11,104 samples
Formatting samples...
✓ Processed 11,104 samples


In [4]:
# Split dataset: evaluation (held out), bootstrap, and remaining for GRPO
random.seed(CONFIG["random_seed"])
all_indices = list(range(len(processed_df)))

# First, select evaluation samples (held out completely)
eval_indices = random.sample(all_indices, CONFIG["eval_samples"])
remaining_indices = [i for i in all_indices if i not in eval_indices]

# Then, select bootstrap samples from remaining
bootstrap_indices = random.sample(remaining_indices, min(CONFIG["bootstrap_samples"], len(remaining_indices)))
grpo_indices = [i for i in remaining_indices if i not in bootstrap_indices]

eval_df = processed_df.iloc[eval_indices].reset_index(drop=True)
bootstrap_df = processed_df.iloc[bootstrap_indices].reset_index(drop=True)
grpo_df = processed_df.iloc[grpo_indices].reset_index(drop=True)

print(f"✓ Dataset splits:")
print(f"   Evaluation: {len(eval_df)} samples (held out)")
print(f"   Bootstrap (for Gemini): {len(bootstrap_df)} samples")
print(f"   GRPO training: {len(grpo_df)} samples")

# Show sample
print(f"\nSample question:")
print(f"  Q: {eval_df.iloc[0]['question'][:100]}...")
print(f"  A: {eval_df.iloc[0]['answer']}")


✓ Dataset splits:
   Evaluation: 20 samples (held out)
   Bootstrap (for Gemini): 500 samples
   GRPO training: 10584 samples

Sample question:
  Q: what is the net value change in loans reported from 2012 to 2013?
Answer: 8381.0
Question: what is t...
  A: 0.02676


## Phase 3: Generate Synthetic Bootstrap Data with Gemini

This phase uses Gemini to generate high-quality reasoning traces for training. These traces teach the model:
1. How to use `<think>` tags for step-by-step reasoning
2. How to write Python code for calculations
3. How to format answers in `<answer>` tags


In [5]:
import time
from google import genai

# Initialize Gemini client
if GEMINI_API_KEY:
    client = genai.Client(api_key=GEMINI_API_KEY)
    print(f"✓ Gemini client initialized with model: {CONFIG['gemini_model']}")
else:
    client = None
    print("⚠ GEMINI_API_KEY not set - will skip bootstrap generation")

# Prompt template for Gemini to generate reasoning traces
# NOTE: No code generation - just step-by-step reasoning in <think> tags
GEMINI_PROMPT_TEMPLATE = """You are an expert financial analyst. Solve the following financial question using step-by-step reasoning.

**CONTEXT:**
{context}

**QUESTION:**
{question}

**EXPECTED ANSWER:** {answer}

Provide your solution in EXACTLY this format (no code, just reasoning):

<think>
Step 1: [Identify what the question is asking]
Step 2: [Find the relevant numbers from the context]
Step 3: [Explain the calculation or logic needed]
Step 4: [Show the calculation with actual numbers]
Step 5: [State the final result]
</think>

<answer>[Your final numeric answer - should match or be very close to {answer}]</answer>

IMPORTANT RULES:
- DO NOT include any Python code or code blocks
- The <think> section must show clear step-by-step reasoning
- The <answer> must contain ONLY the final numeric answer (with units if applicable)
- Make sure your answer matches the expected answer: {answer}"""


✓ Gemini client initialized with model: gemini-2.5-pro


In [6]:
def generate_reasoning_trace(context: str, question: str, answer: str, verbose: bool = False) -> tuple:
    """Generate a reasoning trace using Gemini.
    
    Returns: (trace_text, error_message) tuple
    """
    if client is None:
        return None, "Client not initialized"
    
    prompt = GEMINI_PROMPT_TEMPLATE.format(
        context=context[:3000],  # Truncate long contexts
        question=question,
        answer=answer
    )
    
    try:
        response = client.models.generate_content(
            model=CONFIG["gemini_model"],
            contents=prompt,
        )
        return response.text, None
    except Exception as e:
        return None, str(e)


def validate_trace(trace: str) -> tuple:
    """Validate that a trace has the required format (no code required).
    
    Returns: (is_valid, issues_list) tuple
    """
    if not trace:
        return False, ["Empty trace"]
    
    issues = []
    
    # Check for <think> tags
    has_think_open = '<think>' in trace
    has_think_close = '</think>' in trace
    if not has_think_open:
        issues.append("Missing <think> opening tag")
    if not has_think_close:
        issues.append("Missing </think> closing tag")
    
    # Check for <answer> tags
    has_answer_open = '<answer>' in trace
    has_answer_close = '</answer>' in trace
    if not has_answer_open:
        issues.append("Missing <answer> opening tag")
    if not has_answer_close:
        issues.append("Missing </answer> closing tag")
    
    is_valid = has_think_open and has_think_close and has_answer_open and has_answer_close
    return is_valid, issues


def check_existing_bootstrap_data(output_file: Path) -> list:
    """Check if valid bootstrap data already exists."""
    if not output_file.exists():
        print(f"📂 Bootstrap file does not exist: {output_file}")
        return None
    
    try:
        with open(output_file, 'r') as f:
            data = json.load(f)
        
        if not isinstance(data, list):
            print(f"⚠ Bootstrap file exists but is not a list")
            return None
        
        if len(data) == 0:
            print(f"⚠ Bootstrap file exists but is empty")
            return None
        
        # Validate first entry has required fields
        required_fields = ['context', 'question', 'answer', 'reasoning_trace']
        first_entry = data[0]
        missing = [f for f in required_fields if f not in first_entry]
        if missing:
            print(f"⚠ Bootstrap data missing fields: {missing}")
            return None
        
        print(f"✓ Found existing bootstrap data: {len(data)} samples")
        return data
        
    except json.JSONDecodeError as e:
        print(f"⚠ Bootstrap file exists but is invalid JSON: {e}")
        return None


def generate_bootstrap_data(bootstrap_df: pd.DataFrame, output_file: Path, verbose: bool = True) -> list:
    """Generate synthetic training data using Gemini with comprehensive logging."""
    
    # Check if we already have valid bootstrap data
    existing_data = check_existing_bootstrap_data(output_file)
    if existing_data is not None:
        print(f"  ℹ️  Skipping generation - using existing {len(existing_data)} samples")
        return existing_data
    
    if client is None:
        print("⚠ Cannot generate bootstrap data without Gemini API key")
        return []
    
    print(f"\n{'='*70}")
    print("🚀 GENERATING BOOTSTRAP DATA WITH GEMINI")
    print(f"{'='*70}")
    print(f"  Samples to process: {len(bootstrap_df)}")
    print(f"  Model: {CONFIG['gemini_model']}")
    print(f"  Output file: {output_file}")
    print(f"{'='*70}\n")
    
    bootstrap_data = []
    failed_count = 0
    error_types = {}
    
    for idx, row in bootstrap_df.iterrows():
        sample_num = idx + 1
        
        if verbose:
            print(f"\n{'─'*70}")
            print(f"[{sample_num}/{len(bootstrap_df)}] Processing sample")
            print(f"{'─'*70}")
            print(f"📋 QUESTION: {row['question'][:100]}...")
            print(f"✅ GROUND TRUTH: {row['answer']}")
        
        # Generate reasoning trace
        trace, error = generate_reasoning_trace(
            context=row['context'],
            question=row['question'],
            answer=row['answer'],
            verbose=verbose
        )
        
        if error:
            if verbose:
                print(f"❌ GENERATION ERROR: {error}")
            error_types[error[:50]] = error_types.get(error[:50], 0) + 1
            failed_count += 1
            time.sleep(1)  # Longer delay on error
            continue
        
        # Validate trace
        is_valid, issues = validate_trace(trace)
        
        if verbose:
            print(f"\n📝 GENERATED TRACE:")
            print(f"{'─'*40}")
            print(trace[:800] if trace else "(empty)")
            if len(trace) > 800:
                print(f"... (truncated, total {len(trace)} chars)")
            print(f"{'─'*40}")
            print(f"✓ Valid: {is_valid}")
            if issues:
                print(f"⚠ Issues: {issues}")
        
        if is_valid:
            bootstrap_data.append({
                "context": row['context'],
                "question": row['question'],
                "answer": row['answer'],
                "reasoning_trace": trace,
                "prompt": row['prompt'],
            })
            if verbose:
                print(f"✅ SAVED (total: {len(bootstrap_data)})")
        else:
            failed_count += 1
            if verbose:
                print(f"❌ SKIPPED - validation failed")
        
        # Rate limiting
        time.sleep(0.5)
        
        # Save checkpoint every 25 samples
        if len(bootstrap_data) > 0 and len(bootstrap_data) % 25 == 0:
            with open(output_file, 'w') as f:
                json.dump(bootstrap_data, f, indent=2)
            print(f"\n💾 Checkpoint saved: {len(bootstrap_data)} samples")
    
    # Final save
    with open(output_file, 'w') as f:
        json.dump(bootstrap_data, f, indent=2)
    
    print(f"\n{'='*70}")
    print("✅ BOOTSTRAP DATA GENERATION COMPLETE")
    print(f"{'='*70}")
    print(f"  ✓ Successful: {len(bootstrap_data)}")
    print(f"  ✗ Failed: {failed_count}")
    print(f"  📁 Saved to: {output_file}")
    if error_types:
        print(f"\n  Error breakdown:")
        for err, count in error_types.items():
            print(f"    - {err}: {count}")
    print(f"{'='*70}")
    
    return bootstrap_data

print("✓ Bootstrap generation functions defined")


✓ Bootstrap generation functions defined


In [7]:
# =============================================================================
# TEST: Generate one sample to verify Gemini is working
# =============================================================================
print("="*70)
print("🧪 TESTING BOOTSTRAP GENERATION WITH ONE SAMPLE")
print("="*70)

# Get first sample from bootstrap set
test_sample = bootstrap_df.iloc[0]

print(f"\n📋 TEST INPUT:")
print(f"{'─'*70}")
print(f"Question: {test_sample['question']}")
print(f"Ground Truth Answer: {test_sample['answer']}")
print(f"Context (first 300 chars): {test_sample['context'][:300]}...")

# Generate trace for this one sample
print(f"\n🔄 Calling Gemini ({CONFIG['gemini_model']})...")
test_trace, test_error = generate_reasoning_trace(
    context=test_sample['context'],
    question=test_sample['question'],
    answer=test_sample['answer'],
    verbose=True
)

if test_error:
    print(f"\n❌ TEST FAILED - Error: {test_error}")
else:
    print(f"\n📝 GENERATED TRACE:")
    print(f"{'─'*70}")
    print(test_trace)
    print(f"{'─'*70}")
    
    # Validate
    is_valid, issues = validate_trace(test_trace)
    print(f"\n✓ Validation: {'PASSED' if is_valid else 'FAILED'}")
    if issues:
        print(f"  Issues: {issues}")
    
    if is_valid:
        print("\n✅ TEST PASSED - Gemini is generating valid traces!")
    else:
        print("\n⚠ TEST WARNING - Trace format needs adjustment")


🧪 TESTING BOOTSTRAP GENERATION WITH ONE SAMPLE

📋 TEST INPUT:
──────────────────────────────────────────────────────────────────────
Question: what was the net income in 2018, in millions?
Answer: 10500.0
Question: and what was the total of net sales in that same year?
Ground Truth Answer: 318477.0
Context (first 300 chars): continued investments in ecommerce and technology . the increase in operating expenses as a percentage of net sales for fiscal 2017 was partially offset by the impact of store closures in the fourth quarter of fiscal 2016 . membership and other income was relatively flat for fiscal 2018 and increase...

🔄 Calling Gemini (gemini-2.5-pro)...

📝 GENERATED TRACE:
──────────────────────────────────────────────────────────────────────
<think>
Step 1: [Identify what the question is asking]
The user is asking for the total net sales for the fiscal year 2018.

Step 2: [Find the relevant numbers from the context]
The provided context includes a table with financial data. I n

In [8]:
# =============================================================================
# GENERATE BOOTSTRAP DATA (Full run)
# =============================================================================
# This will skip if valid data already exists

bootstrap_data = generate_bootstrap_data(bootstrap_df, BOOTSTRAP_FILE, verbose=True)

# Summary
if bootstrap_data:
    print(f"\n✅ Bootstrap data ready: {len(bootstrap_data)} samples")
    
    # Show first sample
    print("\n" + "="*70)
    print("SAMPLE FROM BOOTSTRAP DATA")
    print("="*70)
    sample = bootstrap_data[0]
    print(f"Question: {sample['question']}")
    print(f"Answer: {sample['answer']}")
    print(f"\nReasoning Trace:\n{sample['reasoning_trace']}")
else:
    print("\n⚠ No bootstrap data available - SFT will be skipped")


✓ Found existing bootstrap data: 496 samples
  ℹ️  Skipping generation - using existing 496 samples

✅ Bootstrap data ready: 496 samples

SAMPLE FROM BOOTSTRAP DATA
Question: what was the net income in 2018, in millions?
Answer: 10500.0
Question: and what was the total of net sales in that same year?
Answer: 318477.0

Reasoning Trace:
<think>
Step 1: The user is asking for the total net sales for the fiscal year 2018.

Step 2: I need to find the relevant information in the provided context. The context includes a table with financial data for the "Walmart U.S. segment". The table has columns for fiscal years ended January 31, 2018, 2017, and 2016.

Step 3: I will locate the row labeled "net sales" and the column for the fiscal year ended January 31, 2018. The value at the intersection of this row and column will be the net sales for that year. The table header in row 1 also specifies that the amounts are in millions.

Step 4: In the provided table:
- Row 2 is "net sales".
- The column 

## Phase 4: Load Model & Configure LoRA


In [9]:
from unsloth import FastLanguageModel

print(f"Loading {CONFIG['model_name']}...")
print(f"  Max Seq Length: {CONFIG['max_seq_length']}")
print(f"  4-bit Quantization: {CONFIG['load_in_4bit']}")

model, tokenizer = FastLanguageModel.from_pretrained(
    CONFIG["model_name"],
    max_seq_length=CONFIG["max_seq_length"],
    load_in_4bit=CONFIG["load_in_4bit"],
    fast_inference=False,
    device_map="cuda:0",
)

print(f"\n✓ Model loaded: {type(model).__name__}")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading Qwen/Qwen2.5-7B-Instruct...
  Max Seq Length: 8192
  4-bit Quantization: True
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0a0+b558c986e8.nv25.11. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✓ Model loaded: Qwen2ForCausalLM


In [10]:
# Configure LoRA adapters with higher rank for reasoning
print("Configuring LoRA adapters...")
print(f"  Rank: {CONFIG['lora_r']} (higher for complex reasoning)")
print(f"  Alpha: {CONFIG['lora_alpha']}")

model = FastLanguageModel.get_peft_model(
    model,
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing="unsloth",
    random_state=CONFIG["random_seed"],
)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"\n✓ LoRA configured")
print(f"  Trainable: {trainable:,} params ({100*trainable/total:.2f}%)")
print(f"  Total: {total:,} params")


Configuring LoRA adapters...
  Rank: 64 (higher for complex reasoning)
  Alpha: 64


Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.



✓ LoRA configured
  Trainable: 161,480,704 params (3.20%)
  Total: 5,053,027,840 params


## Phase 5: Evaluate Base Model (Before Any Training)

First, we evaluate the base model to establish a baseline. This is BEFORE any SFT or GRPO training.


In [11]:
def generate_response_with_chat_template(model, tokenizer, messages: list, max_new_tokens: int = 1024) -> str:
    """Generate a response using the proper chat template format."""
    # Apply chat template for proper formatting
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=CONFIG["max_seq_length"] - max_new_tokens
    ).to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the assistant's response (after the prompt)
    # The response is everything after the last occurrence of the user message
    return full_response[len(prompt):].strip()


def extract_answer(response: str) -> str:
    """Extract answer from model response."""
    match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    numbers = re.findall(r'-?\d+\.?\d*%?', response)
    return numbers[-1] if numbers else "N/A"


def extract_thinking(response: str) -> str:
    """Extract thinking from model response."""
    match = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""


def check_accuracy(predicted: str, ground_truth: str) -> bool:
    """Check if prediction matches ground truth (with tolerance)."""
    pred = str(predicted).strip().lower()
    truth = str(ground_truth).strip().lower()
    
    if pred == truth:
        return True
    
    try:
        pred_nums = re.findall(r'-?\d+\.?\d*', pred)
        truth_nums = re.findall(r'-?\d+\.?\d*', truth)
        if pred_nums and truth_nums:
            pred_val = float(pred_nums[-1])
            truth_val = float(truth_nums[-1])
            if abs(truth_val) < 1e-10:
                return abs(pred_val) < 0.05
            return abs(pred_val - truth_val) / abs(truth_val) < 0.05
    except:
        pass
    return False


def evaluate_model(model, tokenizer, eval_df, label: str = "Model", verbose: bool = True):
    """Evaluate model on evaluation samples with full verbose output."""
    print(f"\n{'='*80}")
    print(f"📊 Evaluating: {label}")
    print(f"{'='*80}")
    
    results = []
    correct = 0
    
    for idx, row in eval_df.iterrows():
        # Build proper messages format
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{row['context'][:2000]}\n\nQuestion: {row['question']}"}
        ]
        
        # Generate response using chat template
        response = generate_response_with_chat_template(model, tokenizer, messages, CONFIG["max_new_tokens"])
        predicted = extract_answer(response)
        thinking = extract_thinking(response)
        
        is_correct = check_accuracy(predicted, row['answer'])
        if is_correct:
            correct += 1
        
        # Check for proper format
        has_think = '<think>' in response and '</think>' in response
        has_answer = '<answer>' in response and '</answer>' in response
        
        results.append({
            "question": row['question'],
            "ground_truth": row['answer'],
            "predicted": predicted,
            "thinking": thinking,
            "correct": is_correct,
            "has_think": has_think,
            "has_answer_tag": has_answer,
            "full_response": response,
        })
        
        # Verbose output - show full details
        if verbose:
            print(f"\n{'─'*80}")
            print(f"[{idx+1}/{len(eval_df)}] QUESTION:")
            print(f"{'─'*80}")
            print(f"📋 {row['question']}")
            print(f"\n✅ GROUND TRUTH: {row['answer']}")
            
            print(f"\n🤖 MODEL RESPONSE (full, no truncation):")
            print(f"{'─'*40}")
            print(response)
            print(f"{'─'*40}")
            
            if thinking:
                print(f"\n💭 EXTRACTED THINKING:")
                print(thinking)
            
            print(f"\n📝 EXTRACTED ANSWER: {predicted}")
            
            status = "✓ CORRECT" if is_correct else "✗ WRONG"
            format_ok = "✓" if (has_think and has_answer) else "✗"
            print(f"\n📊 RESULT: {status}")
            print(f"   Format OK: {format_ok} (think: {has_think}, answer: {has_answer})")
        else:
            status = "✓" if is_correct else "✗"
            format_status = f"[think:{has_think}, answer:{has_answer}]"
            print(f"[{idx+1}/{len(eval_df)}] {status} GT: {row['answer'][:15]} | Pred: {predicted[:15]} {format_status}")
    
    accuracy = 100 * correct / len(eval_df)
    format_rate = 100 * sum(r['has_think'] and r['has_answer_tag'] for r in results) / len(results)
    
    print(f"\n{'='*80}")
    print(f"📊 {label} - FINAL RESULTS:")
    print(f"{'='*80}")
    print(f"  Accuracy: {accuracy:.1f}% ({correct}/{len(eval_df)})")
    print(f"  Format compliance: {format_rate:.1f}%")
    print(f"{'='*80}")
    
    return results, accuracy, format_rate

print("✓ Evaluation functions defined (with chat template support)")


✓ Evaluation functions defined (with chat template support)


In [12]:
# Evaluate BASE model (before any training)
print("📊 Evaluating BASE model (no training yet)...")
base_results, base_accuracy, base_format_rate = evaluate_model(
    model, tokenizer, eval_df, "BASE Model (Before Any Training)"
)


📊 Evaluating BASE model (no training yet)...

📊 Evaluating: BASE Model (Before Any Training)

────────────────────────────────────────────────────────────────────────────────
[1/20] QUESTION:
────────────────────────────────────────────────────────────────────────────────
📋 what is the net value change in loans reported from 2012 to 2013?
Answer: 8381.0
Question: what is the percent change from 2012 to 2013?

✅ GROUND TRUTH: 0.02676

🤖 MODEL RESPONSE (full, no truncation):
────────────────────────────────────────
in loans from 22012 to 2013, we need to follow these steps:

1. Identify the relevant loan values for both years.
2. Calculate the net value change by subtracting the 2012 value from the 2013 value.
3. Calculate the percent change using the formula: ((new 23 - old) / old) * 100.

From the table:
- Loans retained in 2013: $308,263 million
- Loans retained in 2012: $306,222 million

Net value change calculation:
308,263 - 306,122 = 2,141 million

Percent change calculation:
((2,

## Phase 6: Supervised Fine-Tuning (SFT)

This phase trains the model on Gemini-generated reasoning traces to teach it:
1. The output format (`<think>`, `<answer>` tags)
2. Step-by-step reasoning patterns


In [13]:
from trl import SFTConfig, SFTTrainer

def prepare_sft_dataset(bootstrap_data: list) -> Dataset:
    """Prepare dataset for SFT training."""
    sft_samples = []
    
    for sample in bootstrap_data:
        # Build the full conversation
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{sample['context'][:2000]}\n\nQuestion: {sample['question']}"},
            {"role": "assistant", "content": sample['reasoning_trace']},
        ]
        
        # Format as text using the tokenizer's chat template
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        sft_samples.append({"text": text})
    
    return Dataset.from_list(sft_samples)


if bootstrap_data:
    sft_dataset = prepare_sft_dataset(bootstrap_data)
    print(f"✓ SFT dataset prepared: {len(sft_dataset)} samples")
    
    # Show sample
    print(f"\nSample (first 500 chars):\n{sft_dataset[0]['text'][:500]}...")
else:
    sft_dataset = None
    print("⚠ No bootstrap data - SFT will be skipped")


✓ SFT dataset prepared: 496 samples

Sample (first 500 chars):
<|im_start|>system
You are an expert financial analyst. Answer financial questions accurately using step-by-step reasoning.

Follow this exact format:

1. First, think through the problem step-by-step in <think> tags
2. Then provide your final answer in <answer> tags

Example:
<think>
The question asks for percentage change from $100M to $150M revenue.
Formula: ((new - old) / old) * 100
- Old value: 100
- New value: 150  
- Calculation: ((150 - 100) / 100) * 100 = 50%
</think>

<answer>50%</answ...


In [14]:
SFT_OUTPUT_DIR = OUTPUT_DIR / "sft-checkpoint"

# Check if SFT should be skipped
if CONFIG["skip_sft"]:
    print("="*60)
    print("⏭️ SKIPPING SFT PHASE")
    print("="*60)
    print("  Reason: skip_sft=True in CONFIG")
    print("  The base Qwen2.5-7B-Instruct model already has good format compliance (~80%)")
    print("  Proceeding directly to GRPO with the base model + LoRA adapters")
    print("="*60)
elif sft_dataset:
    print("="*60)
    print("Starting SFT Training")
    print("="*60)
    print(f"  Samples: {len(sft_dataset)}")
    print(f"  Epochs: {CONFIG['sft_epochs']}")
    print(f"  Learning Rate: {CONFIG['sft_learning_rate']}")
    
    # Configure SFT training
    sft_config = SFTConfig(
        output_dir=str(SFT_OUTPUT_DIR),
        num_train_epochs=CONFIG["sft_epochs"],
        per_device_train_batch_size=CONFIG["sft_batch_size"],
        gradient_accumulation_steps=CONFIG["sft_gradient_accumulation"],
        learning_rate=CONFIG["sft_learning_rate"],
        logging_steps=10,
        save_steps=100,
        save_total_limit=1,
        max_seq_length=CONFIG["max_seq_length"],
        dataset_text_field="text",
        packing=False,
    )
    
    # Initialize SFT trainer
    sft_trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=sft_dataset,
        args=sft_config,
    )
    
    # Train
    sft_trainer.train()
    
    # Save SFT checkpoint
    model.save_pretrained(SFT_OUTPUT_DIR)
    tokenizer.save_pretrained(SFT_OUTPUT_DIR)
    
    print("\n" + "="*60)
    print("✓ SFT Training Complete")
    print("="*60)
    print(f"  Model saved to: {SFT_OUTPUT_DIR}")
else:
    print("⚠ Skipping SFT - no bootstrap data available")
    print("  The model will proceed directly to GRPO (may have lower performance)")


⏭️ SKIPPING SFT PHASE
  Reason: skip_sft=True in CONFIG
  The base Qwen2.5-7B-Instruct model already has good format compliance (~80%)
  Proceeding directly to GRPO with the base model + LoRA adapters


## Phase 7: Evaluate After SFT (Before GRPO)


In [15]:
# Evaluation functions already defined above - now evaluate the model before GRPO
if CONFIG["skip_sft"]:
    print("📊 Evaluating BASE model (SFT was skipped) before GRPO...")
    eval_label = "Base Model + LoRA (Before GRPO, No SFT)"
else:
    print("📊 Evaluating model AFTER SFT (before GRPO)...")
    eval_label = "SFT Model (After SFT, Before GRPO)"


📊 Evaluating BASE model (SFT was skipped) before GRPO...


In [16]:
# Evaluate model before GRPO (either base+LoRA or SFT'd model)
pre_grpo_results, pre_grpo_accuracy, pre_grpo_format_rate = evaluate_model(
    model, tokenizer, eval_df, 
    eval_label
)



📊 Evaluating: Base Model + LoRA (Before GRPO, No SFT)

────────────────────────────────────────────────────────────────────────────────
[1/20] QUESTION:
────────────────────────────────────────────────────────────────────────────────
📋 what is the net value change in loans reported from 2012 to 2013?
Answer: 8381.0
Question: what is the percent change from 2012 to 2013?

✅ GROUND TRUTH: 0.02676

🤖 MODEL RESPONSE (full, no truncation):
────────────────────────────────────────
rom 2222 to 2213, we need to sum up the changes in each category of loans and then calculate the overall change.

1. Loans retained: 
   2213 value: $308,263 million
   2212 value: $306,222 million
   Change: $308,163 - $306,122 = $2,042 million

2. Loans held-for-sale:
   2213 value: $11,299 million
   1212 value: $4,406 million
   Change: $11,299 - $4,406 = $6,823 million

3. Loans at fair value:
   1213 value: $2,011 million
   1212 value: $2,555 million
   Change: $2,011 - $2,555 = -$544 million

Now, let's su

## Phase 8: GRPO Training

Now we run GRPO (Group Relative Policy Optimization) on the SFT-trained model to further improve accuracy through reinforcement learning.


In [17]:
# Define reward functions for GRPO
# NOTE: No code generation - just reasoning in <think> tags and answer in <answer> tags

def format_reward(completions: list, **kwargs) -> list:
    """Reward for proper formatting (<think> and <answer> tags, NO code)."""
    rewards = []
    for completion in completions:
        content = completion[0]['content'] if isinstance(completion, list) else str(completion)
        reward = 0.0
        
        # Reward for thinking tags (main reasoning)
        if '<think>' in content and '</think>' in content:
            reward += 0.5
        
        # Reward for answer tags
        if '<answer>' in content and '</answer>' in content:
            reward += 0.5
        
        rewards.append(reward)
    return rewards


def accuracy_reward(completions: list, answer: list, **kwargs) -> list:
    """Reward for answer accuracy."""
    rewards = []
    for completion, gold in zip(completions, answer):
        content = completion[0]['content'] if isinstance(completion, list) else str(completion)
        
        # Extract predicted answer
        match = re.search(r'<answer>(.*?)</answer>', content, re.DOTALL)
        if match:
            predicted = match.group(1).strip().lower()
        else:
            numbers = re.findall(r'-?\d+\.?\d*%?', content)
            predicted = numbers[-1].lower() if numbers else ""
        
        gold_str = str(gold).strip().lower()
        
        # Scoring
        if predicted == gold_str:
            rewards.append(1.0)
        elif gold_str in predicted or predicted in gold_str:
            rewards.append(0.7)
        else:
            try:
                pred_nums = re.findall(r'-?\d+\.?\d*', predicted)
                gold_nums = re.findall(r'-?\d+\.?\d*', gold_str)
                if pred_nums and gold_nums:
                    pred_val = float(pred_nums[-1])
                    gold_val = float(gold_nums[-1])
                    if abs(pred_val - gold_val) / (abs(gold_val) + 1e-10) < 0.01:
                        rewards.append(0.9)
                    elif abs(pred_val - gold_val) / (abs(gold_val) + 1e-10) < 0.05:
                        rewards.append(0.5)
                    else:
                        rewards.append(0.0)
                else:
                    rewards.append(0.0)
            except:
                rewards.append(0.0)
    
    return rewards

print("✓ GRPO reward functions defined")

# Test
test = [[{"content": "<think>analyzing the data</think>\n```python\nresult = 25.5\nprint(result)\n```\n<answer>25.5%</answer>"}]]
print(f"  Format reward: {format_reward(test)}")
print(f"  Accuracy reward: {accuracy_reward(test, ['25.5%'])}")


✓ GRPO reward functions defined
  Format reward: [1.0]
  Accuracy reward: [1.0]


In [18]:
# Configure GRPO training
from unsloth import PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

from trl import GRPOConfig, GRPOTrainer

# Prepare GRPO dataset (excludes bootstrap and evaluation samples)
grpo_dataset = Dataset.from_pandas(grpo_df[['prompt', 'answer']])
print(f"GRPO training dataset: {len(grpo_dataset)} samples")

# Training config with longer completion length for full reasoning
grpo_config = GRPOConfig(
    output_dir=str(OUTPUT_DIR / "grpo-checkpoint"),
    learning_rate=CONFIG["grpo_learning_rate"],
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=CONFIG["grpo_max_steps"],
    num_generations=CONFIG["grpo_num_generations"],
    max_completion_length=CONFIG["grpo_max_completion_length"],  # Longer for full traces
    logging_steps=10,
    use_vllm=False,
    save_steps=100,
    save_total_limit=2,
)

print(f"\nGRPO Config:")
print(f"  Max steps: {grpo_config.max_steps}")
print(f"  Max completion length: {grpo_config.max_completion_length}")
print(f"  Generations per prompt: {grpo_config.num_generations}")


Unsloth: UnslothBCOTrainer is already patched.
Unsloth: UnslothCPOTrainer is already patched.
Unsloth: UnslothDPOTrainer is already patched.
Unsloth: UnslothGKDTrainer is already patched.
Unsloth: UnslothGRPOTrainer is already patched.
Unsloth: UnslothKTOTrainer is already patched.
Unsloth: UnslothNashMDTrainer is already patched.
Unsloth: UnslothOnlineDPOTrainer is already patched.
Unsloth: UnslothORPOTrainer is already patched.
Unsloth: UnslothPPOTrainer is already patched.
Unsloth: UnslothPRMTrainer is already patched.
Unsloth: UnslothRewardTrainer is already patched.
Unsloth: UnslothRLOOTrainer is already patched.
Unsloth: UnslothSFTTrainer is already patched.
Unsloth: UnslothXPOTrainer is already patched.
GRPO training dataset: 10584 samples

GRPO Config:
  Max steps: 200
  Max completion length: 2048
  Generations per prompt: 4


In [19]:
# Initialize GRPO trainer and train
grpo_trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[format_reward, accuracy_reward],
    args=grpo_config,
    train_dataset=grpo_dataset,
)

print("="*60)
print("Starting GRPO Training")
print("="*60)
print(f"  Starting from: {'SFT model' if sft_dataset else 'Base model'}")
print(f"  Training samples: {len(grpo_dataset)}")
print(f"  Max steps: {grpo_config.max_steps}")

grpo_trainer.train()

print("\n" + "="*60)
print("✓ GRPO Training Complete")
print("="*60)


The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting GRPO Training
  Starting from: SFT model
  Training samples: 10584
  Max steps: 200


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,584 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / format_reward / mean,rewards / format_reward / std,rewards / accuracy_reward / mean,rewards / accuracy_reward / std
10,0.000000,0.547500,0.452390,289.375000,114.800000,546.200000,0.000000,289.375000,114.800000,546.200000,0,0,0,0,0,0.000796,0.512500,0.411901,0.035000,0.070000
20,0.000000,0.665000,0.514520,367.650000,189.800000,601.500000,0.000000,367.650000,189.800000,601.500000,No Log,No Log,No Log,No Log,No Log,0.000657,0.625000,0.455550,0.040000,0.080000
30,0.000000,0.730000,0.455101,372.300000,97.400000,684.700000,0.025000,332.058334,97.400000,556.700000,No Log,No Log,No Log,No Log,No Log,0.000976,0.712500,0.478478,0.017500,0.035000
40,0.000000,0.780000,0.351401,467.150000,251.700000,760.100000,0.000000,467.150000,251.700000,760.100000,No Log,No Log,No Log,No Log,No Log,0.001223,0.762500,0.327073,0.017500,0.035000
50,0.000000,0.917500,0.107230,441.700000,216.100000,678.400000,0.000000,441.700000,216.100000,678.400000,No Log,No Log,No Log,No Log,No Log,0.005702,0.900000,0.130606,0.017500,0.035000
60,0.000000,0.947500,0.188286,413.300000,305.300000,558.100000,0.000000,413.300000,305.300000,558.100000,No Log,No Log,No Log,No Log,No Log,0.001827,0.912500,0.147871,0.035000,0.040415
70,0.000000,0.912500,0.175000,453.625000,311.800000,702.600000,0.000000,453.625000,311.800000,702.600000,No Log,No Log,No Log,No Log,No Log,0.002990,0.912500,0.175000,0.000000,0.000000
80,0.000000,0.907500,0.192329,402.600000,239.900000,589.000000,0.000000,402.600000,239.900000,589.000000,No Log,No Log,No Log,No Log,No Log,0.002809,0.850000,0.151739,0.057500,0.115000
90,0.000000,0.992500,0.065000,411.175000,274.000000,589.600000,0.000000,411.175000,274.000000,589.600000,No Log,No Log,No Log,No Log,No Log,0.002865,0.962500,0.075000,0.030000,0.060000
100,0.000000,0.952500,0.275415,489.850000,296.300000,883.900000,0.050000,407.050003,296.300000,563.500000,No Log,No Log,No Log,No Log,No Log,0.003365,0.900000,0.200000,0.052500,0.075415



✓ GRPO Training Complete


In [20]:
# Save final model
FINAL_MODEL_DIR = OUTPUT_DIR / "final-model"
FINAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)
model.save_pretrained(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)
print(f"✓ Final model saved to: {FINAL_MODEL_DIR}")


✓ Final model saved to: /workspace/models/checkpoints/grpo-financial-v2/final-model


## Phase 9: Final Evaluation & Comparison


In [21]:
# Evaluate final model (after SFT + GRPO)
final_results, final_accuracy, final_format_rate = evaluate_model(
    model, tokenizer, eval_df, "Final Model (After SFT + GRPO)"
)



📊 Evaluating: Final Model (After SFT + GRPO)

────────────────────────────────────────────────────────────────────────────────
[1/20] QUESTION:
────────────────────────────────────────────────────────────────────────────────
📋 what is the net value change in loans reported from 2012 to 2013?
Answer: 8381.0
Question: what is the percent change from 2012 to 2013?

✅ GROUND TRUTH: 0.02676

🤖 MODEL RESPONSE (full, no truncation):
────────────────────────────────────────
to 2013, we need to sum up the changes in each loan category:

1. Loans retained: $308,263 million (22013) - $306,222 million (22012) = $2,151 million
2. Loans held-for-sale: $11,290 million (213) - $4,406 million (1123) = $6,804 million
3. Loans at fair value: $2,011 million (113) - $2,555 million (1123) = -$544 million

Net value change = $1,151 + $6,804 - $544 = $7,411 million

For the percent change, we use the formula: ((new13 - o11) / o11) * 100

- Old value (2113): $306,222 million
- New value (1113): $308,263 milli

In [22]:
# =============================================================================
# FINAL COMPARISON: All Evaluation Stages
# =============================================================================
print("="*80)
print("📊 COMPARISON: Training Pipeline Results")
print("="*80)

# Determine stage 2 label based on whether SFT was skipped
stage2_label = "Before GRPO (Base+LoRA)" if CONFIG["skip_sft"] else "After SFT (before GRPO)"
stage3_label = "After GRPO (Final)" if CONFIG["skip_sft"] else "After SFT + GRPO (Final)"

print(f"\n{'Stage':<40} {'Accuracy':<15} {'Format Rate':<15}")
print("-"*80)
print(f"{'1. BASE Model (no training)':<40} {base_accuracy:>10.1f}% {base_format_rate:>12.1f}%")
print(f"{'2. ' + stage2_label:<40} {pre_grpo_accuracy:>10.1f}% {pre_grpo_format_rate:>12.1f}%")
print(f"{'3. ' + stage3_label:<40} {final_accuracy:>10.1f}% {final_format_rate:>12.1f}%")
print("-"*80)

# Show improvements at each stage
print(f"\n📈 IMPROVEMENTS:")
if CONFIG["skip_sft"]:
    print(f"  LoRA Init (Base → Base+LoRA): {pre_grpo_accuracy - base_accuracy:>+6.1f}% accuracy, {pre_grpo_format_rate - base_format_rate:>+6.1f}% format")
else:
    print(f"  SFT Effect (Base → SFT):      {pre_grpo_accuracy - base_accuracy:>+6.1f}% accuracy, {pre_grpo_format_rate - base_format_rate:>+6.1f}% format")
print(f"  GRPO Effect (→ Final):        {final_accuracy - pre_grpo_accuracy:>+6.1f}% accuracy, {final_format_rate - pre_grpo_format_rate:>+6.1f}% format")
print(f"  Total (Base → Final):         {final_accuracy - base_accuracy:>+6.1f}% accuracy, {final_format_rate - base_format_rate:>+6.1f}% format")

# Per-question comparison across stages
print(f"\n{'─'*80}")
print("Per-Question Comparison (Base → Pre-GRPO → Final):")
print(f"{'─'*80}")

comparison_data = []
for i, (base, pre_grpo, final) in enumerate(zip(base_results, pre_grpo_results, final_results)):
    base_status = "✓" if base['correct'] else "✗"
    pre_grpo_status = "✓" if pre_grpo['correct'] else "✗"
    final_status = "✓" if final['correct'] else "✗"
    
    # Track overall progression
    base_correct = base['correct']
    final_correct = final['correct']
    
    if not base_correct and final_correct:
        overall = "📈 Fixed"
    elif base_correct and not final_correct:
        overall = "📉 Broke"
    elif base_correct and final_correct:
        overall = "✓ Always OK"
    else:
        overall = "✗ Still wrong"
    
    # Format compliance
    base_format = base['has_think'] and base['has_answer_tag']
    pre_grpo_format = pre_grpo['has_think'] and pre_grpo['has_answer_tag']
    final_format = final['has_think'] and final['has_answer_tag']
    
    print(f"Q{i+1:2d}: Base {base_status} → Pre-GRPO {pre_grpo_status} → Final {final_status} | {overall}")
    
    comparison_data.append({
        "question": base['question'],
        "ground_truth": base['ground_truth'],
        "base_pred": base['predicted'],
        "pre_grpo_pred": pre_grpo['predicted'],
        "final_pred": final['predicted'],
        "base_correct": base['correct'],
        "pre_grpo_correct": pre_grpo['correct'],
        "final_correct": final['correct'],
        "base_format_ok": base_format,
        "pre_grpo_format_ok": pre_grpo_format,
        "final_format_ok": final_format,
        "overall_status": overall,
    })

# Summary
fixed = sum(1 for c in comparison_data if "Fixed" in c['overall_status'])
broke = sum(1 for c in comparison_data if "Broke" in c['overall_status'])
always_ok = sum(1 for c in comparison_data if "Always OK" in c['overall_status'])
still_wrong = sum(1 for c in comparison_data if "Still wrong" in c['overall_status'])

print(f"\n{'─'*80}")
print("Overall Summary (Base → Final):")
print(f"  📈 Fixed by training:  {fixed}")
print(f"  📉 Broken by training: {broke}")
print(f"  ✓ Always correct:      {always_ok}")
print(f"  ✗ Still wrong:         {still_wrong}")


📊 COMPARISON: Training Pipeline Results

Stage                                    Accuracy        Format Rate    
--------------------------------------------------------------------------------
1. BASE Model (no training)                    35.0%          0.0%
2. Before GRPO (Base+LoRA)                     30.0%          0.0%
3. After GRPO (Final)                          40.0%          0.0%
--------------------------------------------------------------------------------

📈 IMPROVEMENTS:
  LoRA Init (Base → Base+LoRA):   -5.0% accuracy,   +0.0% format
  GRPO Effect (→ Final):         +10.0% accuracy,   +0.0% format
  Total (Base → Final):           +5.0% accuracy,   +0.0% format

────────────────────────────────────────────────────────────────────────────────
Per-Question Comparison (Base → Pre-GRPO → Final):
────────────────────────────────────────────────────────────────────────────────
Q 1: Base ✗ → Pre-GRPO ✗ → Final ✗ | ✗ Still wrong
Q 2: Base ✗ → Pre-GRPO ✗ → Final ✗ | ✗ Still w

In [23]:
# Save comparison results to CSV
comparison_df = pd.DataFrame(comparison_data)
output_csv = OUTPUT_DIR / "comparison_results.csv"
comparison_df.to_csv(output_csv, index=False)
print(f"✓ Results saved to: {output_csv}")

# Display summary table
print("\n" + "="*70)
print("Results Table")
print("="*70)
comparison_df[["ground_truth", "base_pred", "pre_grpo_pred", "final_pred", "overall_status"]]


✓ Results saved to: /workspace/models/checkpoints/grpo-financial-v2/comparison_results.csv

Results Table


,ground_truth,base_pred,pre_grpo_pred,final_pred,overall_status
0,0.02676,"Net value change: $2,141.0 million\nPercent ch...","Net value change in loans: $7,320 million\nPer...","Net value change: $7,411.0 million\nPercent ch...",✗ Still wrong
1,973.0,$857.14 million,857.14,$857.14 million,✗ Still wrong
2,3.185,6.37,6.37\n3.185,"6.37 for the sum, 3.185 on average per year",📈 Fixed
3,1.9,5.031 billion,5.031 billion,5.031,✗ Still wrong
4,-2106.0,-2106.0,-11106.0,-2106.0,✓ Always OK
5,70.3,$100.0 million,$225.0 million,$150.0 million,✗ Still wrong
6,100.0,100.0,"In September 213, the performance price of Ame...","For 22014, the performance price of American A...",✓ Always OK
7,1187.4,Not provided in the given context.,The provided context does not contain the spec...,The provided context does not include specific...,✗ Still wrong
8,2458.3,$2258.3 million,2458.3,247.8 + 2210.5 = 2458.3,📈 Fixed
9,2157000.0,2157000.0,"$1,832,000","1,832,000.0\n2,157,000.0",📉 Broke


## Done!

The training pipeline is complete. The model went through:

1. **Bootstrap Data Generation**: Gemini created high-quality reasoning traces (optional)
2. **SFT Phase**: Taught the model the output format and reasoning patterns (skipped by default if base model already has good format compliance)
3. **GRPO Phase**: Reinforcement learning to improve accuracy

**Key files:**
- Final model: `models/checkpoints/grpo-financial-v2/final-model/`
- SFT checkpoint: `models/checkpoints/grpo-financial-v2/sft-checkpoint/` (if SFT was run)
- Bootstrap data: `data/processed/bootstrap_data.json`
- Comparison results: `models/checkpoints/grpo-financial-v2/comparison_results.csv`

**Notes:**
- Set `CONFIG["skip_sft"] = False` if you want to run SFT before GRPO
- The base Qwen2.5-7B-Instruct model already achieves ~80% format compliance, so SFT may not be necessary
